In [4]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [5]:
import os,sys
sys.path.append('./misc/lib/python3.7/site-packages')

import math
import numpy as np
import requests
import ipywidgets as widgets
%matplotlib notebook
import matplotlib.pyplot as plt
import IPython.display as Display
from IPython.display import display, display_markdown
from ipywidgets import Layout
from pathlib import Path

import parmed as pmd
import re

import io

from scipy.ndimage import gaussian_filter


np.set_printoptions(precision=8)
np.set_printoptions(suppress=True)


HTMLButtonPrompt = '''<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a href="{link}" target="_blank" >
<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning" style="width:150px; background-color:#CCCCCC; font-size:10pt; color:black">{text}</button>
</a>
</body>
</html>
'''   

# Width of elements in site parameter grid
grid_width = '2.5cm'
ggap = '0.5cm'
grid_total_width = '12cm'

forbidden_strings = ["..", "/", "\\", " ", "~"]

In [6]:
sys.path.append('./main/')

import pigment
import tresp
import pdc
import nsd
import phutil

# Set data storage directory
try:
    DATADIR = os.path.relpath(os.environ['DATADIR'])
except:
    DATADIR = './data'
    phutil.build_data_folders(DATADIR)

In [7]:
#################################################################
# Model load dialogue
#################################################################

param_drop = widgets.Dropdown(
    options=[],
    description='',
    disabled=False,
    layout = Layout(width='5cm')
)

refresh_button = widgets.Button(
    description='Refresh List',
    layout=Layout(width='2.5cm')
)

load_button = widgets.Button(
    description='Load Model',
    layout=Layout(width='2.5cm')
)


def refresh_onclick(b):
    out = !{"ls " + DATADIR + "/exc/"}
    prefList = []
    for line in out:
        if line[-4:]=='.exc':
            prefList.append(line[:-4])
    param_drop.options = prefList
    if len(prefList)>0:
        param_drop.value = prefList[0]

refresh_button.on_click(refresh_onclick)

def get_types():
    
    global Names
    
    TypeList = []
    TypeCount = []
    TypeNdcs = []
    for n in range(0, len(Names)):
        name = Names[n]
        if name[0:3] not in TypeList:
            TypeList.append(name[0:3])
            TypeCount.append(1)
            TypeNdcs.append([n])
            
        else:
            ndx = TypeList.index(name[0:3])
            TypeCount[ndx] = TypeCount[ndx] + 1
            TypeNdcs[ndx].append(n)
            
    return TypeList, TypeCount, TypeNdcs
            

def load_button_onclick(b):
    global Coups
    global Freqs
    global Dips
    global Rots
    global Names
    global PigBox
    global FreqBoxList
    global PigBoxList
    global FWHMBoxList
    
    # If no parameter files are available, abort
    if len(param_drop.options)==0:
        return 0
    
    prefix = param_drop.value
    
    NameString = ''
    with open(DATADIR + "/exc/" + prefix + '.exc/names.txt') as file:
        for line in file:
            # We keep the first non-comment line as the name string
            if line[0]!='#':
                NameString = line[:-1]
                break
                
    if len(NameString)==0:
        print('Error reading name file. Aborting load parameters.')
        return 0
    
    # Set list of pigment names
    Names = []
    for name in NameString.strip().split(" "):
        Names.append(name)
        
    TypeList, TypeCount, TypeNdcs = get_types()
    text = 'Loaded folder ' + prefix + '.exc with: '
    for typ in TypeList:
        text += str(TypeCount[TypeList.index(typ)]) + ' ' + typ + ' pigments, '
    text = text[:-2]
    setmsg(text)
        
    vib_cb_onclick(0)
    phonon_cb_onclick(0)
    
    Coups = np.loadtxt(DATADIR + "/exc/" + prefix + '.exc/coups.txt')
    Freqs = np.loadtxt(DATADIR + "/exc/" + prefix + '.exc/freqs.txt')
    dlens = np.loadtxt(DATADIR + "/exc/" + prefix + '.exc/diplengths.txt')
    Dips = np.loadtxt(DATADIR + "/exc/" + prefix + '.exc/dips.txt')
    Rots = np.loadtxt(DATADIR + "/exc/" + prefix + '.exc/rots.txt')
    
    # Reshape all variables. This ensures that the data types
    # are treated consistently, even if only a single oscillator
    # is present. 
    Nsites = len(Names)
    Coups.shape = (Nsites,Nsites)
    Freqs.shape = (Nsites,)
    dlens.shape = (Nsites,)
    Dips.shape = (Nsites,3)
    
    # Normalize Dipoles and rescale by dlens
    for p in range(0, np.shape(Dips)[0]):
        if dlens[p]!=0:
            Dips[p,:] *= dlens[p] / np.linalg.norm(Dips[p,:])
        
    # Rescale Rots by dipole strength
    Rots = Rots*np.outer(dlens,dlens)
        
    LblList = []
    FreqBoxList = []
    FWHMBoxList = []
    PigBoxList = []
    for n in range(0, len(Freqs)):
        LblList.append(widgets.Label(value=Names[n], layout=Layout(width=grid_width)))
        
        FreqBoxList.append(
            widgets.BoundedFloatText(
                value=Freqs[n],
                disabled=False,
                min=0.0,
                max=20000.0,
                step = 1.0,
                layout=Layout(width=grid_width)
            ))

        FWHMBoxList.append(widgets.BoundedFloatText(
                value=200.0,
                min=0.0,
                max=2000.0,
                width='1cm',
                disabled=False, 
                step = 1.0,
                layout=Layout(width=grid_width)
            ))
    
    for n in range(0, len(Freqs)):
        PigBoxList.append(widgets.Box([LblList[n], FreqBoxList[n], FWHMBoxList[n]], layout=Layout(flex='row', grid_gap=ggap)))
    PigBox.children = [DisorderBox, FreqDefBox, FWHMDefBox, PigHead] + PigBoxList + [PigDescript]
    calc_button.disabled = False
    specfile.value = param_drop.value
    
load_button.on_click(load_button_onclick)

refresh_onclick(0)
Coups = []
Freqs = []
Dips = []
Names = []



#################################################################
# Spectrum window setup
#################################################################


def format_coord(x, y):
    #return 'x=%.1f, y=%1.2f'%(x, y)
    return ''


def build_axes():
    specax.set_yticks([])
    specax.set_xlabel('$\\bar\\nu$ (cm$^{-1}$)')
    specax2.xaxis.tick_top()
    specax2.set_xlabel('$\\lambda$ (nm)')
    specax2.format_coord = format_coord
    update_ax2()
    plt.tight_layout()
    plt.show()

def update_ax2():
    specax2.set_xlim(specax.get_xlim())
    xticks2 = (1.0e7)/(specax.get_xticks())
    dlambda = -np.mean(np.diff(xticks2))
    ndec = max(-int(np.ceil(np.log10(dlambda))), 0)
    fmt = "{:." + str(ndec) + "f}"
    xticklabels2 = []
    for tick in xticks2:
        xticklabels2.append(fmt.format(tick))
    
    specax2.set_xticks((1.0e7)/xticks2)
    specax2.set_xticklabels(xticklabels2)
    specax2.set_xlim(specax.get_xlim())
    plt.tight_layout()
    
    
def update_spec_lims():
    
    lines = [abs_line, cd_line, flu_line, abx_line, cdx_line, flx_line]
    myinf = 1e+100
    mny = myinf
    mxy = -myinf
    for line in lines:
        if line.get_visible()==True:
            line.set_ydata(norm_spec(line.get_ydata()))
            mny = min(mny, np.min(line.get_ydata()))
            mxy = max(mxy, np.max(line.get_ydata()))
    if (mny<myinf) and (mxy>-myinf) and (mny!=mxy):
        specax.set_ylim(mny, mxy)
        
    mnx = len(vaxis) - 1
    mxx = 0
    cutoff = (1e-3)*np.max((np.abs(mxy), np.abs(mny)))
    for line in lines:
        if line.get_visible()==True:
            ndcs = np.where(np.abs(line.get_ydata())>cutoff)[0]
            if len(ndcs>0):
                mnx = min(mnx, np.min(ndcs))
                mxx = max(mxx, np.max(ndcs))
    if mxx>mnx:
        specax.set_xlim(vaxis[mnx], vaxis[mxx])
        update_ax2()
    plt.tight_layout()

        
vaxis = np.arange(9000, 20000, 1, dtype=float)

specout = widgets.Output(layout=Layout(width='4.25in',
                                      height='4.25in'))



#################################################################
# Spectrum calculation setup
#################################################################


def build_sspec(T, ptype):
    
    def nbar(v, T):
        h = (6.626068e-34) #J*s
        ccm = 2.9979e10 # cm/s
        kBT = (1.3806503e-23)*T

        val = np.zeros(np.shape(v))
        ndcs = np.where(v>0)
        efac = np.exp(-h*v[ndcs]*ccm/kBT)
        val[ndcs] = efac/(1 - efac)
        return val

    
    svaxis = vaxis.copy() - vaxis[0]
    
    vib_prof = np.zeros(np.shape(vaxis))
    if vib_cb.value==True:
        
        if ptype.lower()=='cla':
            fname = 'misc/LocalVibs/' + cla_vib_drop.value + '.vib'
            Svib = cla_vib_s.value
        elif ptype.lower()=='clb':
            fname = 'misc/LocalVibs/' + clb_vib_drop.value + '.vib'
            Svib = clb_vib_s.value
        elif ptype.lower()=='bca':
            fname = 'misc/LocalVibs/' + bca_vib_drop.value + '.vib'
            Svib = bca_vib_s.value
        else:
            fname = 'misc/LocalVibs/' + def_vib_drop.value + '.vib'
            Svib = def_vib_s.value
            
        # Load vibrational modes
        modes = np.loadtxt(fname)
        
        # Normalize by desired overal Svib
        vibs_norm_fac = Svib / np.sum(modes[:,1])
        
        for mode in modes:

            # Nearest frequency index 
            ndx = np.argmin(np.abs(vaxis-vaxis[0]-mode[0]))

            # Increase value by S-factor for this mode
            vib_prof[ndx] += mode[1]*vibs_norm_fac
    
    phon_prof = np.zeros(np.shape(vaxis))
    if phonon_cb.value==True:
        
        if ptype.lower()=='cla':
            phon_prof = get_spd(cla_phonon_drop.value)
            Sphon = cla_phonon_s.value
        elif ptype.lower()=='clb':
            phon_prof = get_spd(clb_phonon_drop.value)
            Sphon = clb_phonon_s.value
        elif ptype.lower()=='bca':
            phon_prof = get_spd(bca_phonon_drop.value)
            Sphon = bca_phonon_s.value
        else:
            phon_prof = get_spd(def_phonon_drop.value)
            Sphon = def_phonon_s.value
            
        # Normalize. Note that we normalize the *sum*, 
        # not the integral, since FFT also computes the 
        # Fourier transform as a sum rather than integral. 
        phon_prof /= np.sum(phon_prof)
        phon_prof *= Sphon
    
    prof = phon_prof + vib_prof
    if T==0:
        JT = prof + vib_prof
    else:
        JT = prof*(1.0 + nbar(vaxis-vaxis[0],T))
        JT += prof[::-1]*nbar(vaxis-vaxis[0],T)[::-1]
    
    GT = np.fft.fft(JT)
    sspec = np.real(np.fft.ifft(np.exp(GT-GT[0])))
    
    # Shift to center of spectrum so as to avoid shifting in convolutions
    return np.fft.fftshift(sspec)

import datetime
def calculate_spec():
    
    tic = datetime.datetime.now()
    
    hplanck = (6.626068e-34) #J*s
    kB = 1.3806503e-23 #J/K
    ccm = 2.9979e10 # cm/s
        
    global progress_bar
    global Names
    
    Nsites = len(Freqs)
    Sigmas = np.zeros((Nsites,))
    Temp = float(temp_slider.value)
    
    # Check how many different pigment types there are
    TypeList, TypeCount, TypeNdcs = get_types()
    
    # We'll build separate spectra for each pigment type
    # and then add together at the end. 
    ASpecs = np.zeros((len(vaxis), len(TypeList))) # absorption
    FSpecs = np.zeros((len(vaxis), len(TypeList))) # emission
    CSpecs = np.zeros((len(vaxis), len(TypeList))) # CD
    
    # If adding phonons or local vibrations, buld a 
    # single-site spectrum for each pigment type
    if phonon_cb.value==True or vib_cb.value==True:
        SSpecs = np.zeros((len(vaxis), len(TypeList)))
        for n in range(0, len(TypeList)):
            SSpecs[:,n] = build_sspec(Temp, TypeList[n])
        
    if disorder_cb.value==True:
        Nreps = Nits
        for n in range(0, Nsites):
            Sigmas[n] = FWHMBoxList[n].value/2.355
    else:
        Nreps = int(1)
    
    Ham0 = Coups + np.diag(Freqs)
    
    Nplot = 1000
    SIGMA = float(gauss_fwhm)/2.355
    progress_bar.max = Nreps
    for n in range(0, Nreps):
        dfvec = np.zeros((Nsites,))
        for p in range(0, Nsites):
            dfvec[p] = np.random.normal(0, Sigmas[p])
            
        Ham = Ham0 + np.diag(dfvec)
        eVals,eVecs = np.linalg.eigh(Ham)
        eMu = np.matmul(np.transpose(eVecs), Dips)
        eRot = np.transpose(eVecs)@Rots@eVecs
        
        # Fluorescence updates
        boltfac = np.exp((-hplanck*(eVals-np.mean(eVals))*ccm)/(kB*Temp)) #boltzman factor for each eigenstate 
        boltfacsum = np.sum(boltfac) #partition function/normalizing factor
        pop = boltfac/boltfacsum #normalized populations for eigenstates
        
        # Loop through excitonic states and add contributions
        for p in range(0,Nsites):
            ndx = np.where(np.abs(vaxis-eVals[p])==np.min(np.abs(vaxis-eVals[p])))[0][0]
            
            aval = np.linalg.norm(eMu[p])**2/Nsites #mu is a 3 element vector
            cval = eRot[p,p]/Nsites #mu is a 3 element vector
            fval = pop[p]*np.linalg.norm(eMu[p])**2#/Nsites
            
            # Contribution from each type is proportional to the summed
            # populations of all pigments of this type to exciton p. 
            for t in range(0, len(TypeList)):
                tndcs = TypeNdcs[t]
                tcont = np.sum(eVecs[tndcs,p]**2)
                ASpecs[ndx,t] += tcont*aval
                CSpecs[ndx,t] += tcont*cval
                FSpecs[ndx,t] += tcont*fval
            
        progress_bar.value = n + 1
        
        # Every Nplot cycles we update the plot
        if (n+1)%Nplot==0:
            
            abs_tmp = np.zeros((len(vaxis)))
            cd_tmp = np.zeros((len(vaxis)))
            flu_tmp = np.zeros((len(vaxis)))
            
            if phonon_cb.value==True or vib_cb.value==True:
                for t in range(0, len(TypeList)):
                    abs_tmp += np.convolve(ASpecs[:,t], SSpecs[:,t], 'same')
                    cd_tmp += np.convolve(CSpecs[:,t], SSpecs[:,t], 'same')
                    flu_tmp += np.convolve(FSpecs[:,t], SSpecs[::-1,t], 'same')
                
            # If homogeneous filter is in use
            if gauss_cb.value==True:
                abs_tmp = gaussian_filter(abs_tmp, sigma=SIGMA).copy()
                cd_tmp = gaussian_filter(cd_tmp, sigma=SIGMA).copy()
                flu_tmp = gaussian_filter(flu_tmp, sigma=SIGMA).copy()

            abs_line.set_ydata(abs_tmp)
            cd_line.set_ydata(cd_tmp)
            flu_line.set_ydata(flu_tmp)
                
            update_spec_lims()
            specfig.canvas.draw()
            specfig.canvas.flush_events()
    
    abs_tmp = np.zeros((len(vaxis)))
    cd_tmp = np.zeros((len(vaxis)))
    flu_tmp = np.zeros((len(vaxis)))
    if phonon_cb.value==True or vib_cb.value==True:
        for t in range(0, len(TypeList)):
            abs_tmp += np.convolve(ASpecs[:,t], SSpecs[:,t], 'same')
            cd_tmp += np.convolve(CSpecs[:,t], SSpecs[:,t], 'same')
            flu_tmp += np.convolve(FSpecs[:,t], SSpecs[::-1,t], 'same')
            
    if gauss_cb.value==True:
        abs_tmp = gaussian_filter(abs_tmp, sigma=SIGMA).copy()
        cd_tmp = gaussian_filter(cd_tmp, sigma=SIGMA).copy()
        flu_tmp = gaussian_filter(flu_tmp, sigma=SIGMA).copy()
        
    abs_line.set_ydata(abs_tmp)
    cd_line.set_ydata(cd_tmp)
    flu_line.set_ydata(flu_tmp)
    update_spec_lims()
    
    toc = datetime.datetime.now()
    
    setmsg('Computation time: ' + str(toc-tic))
    return



##########################################################
## Temperature Slider & Buttons
##########################################################

temp_slider = widgets.IntSlider(
    value=300,
    min=1, # min value
    max=400, # max value
    step=1, # step size = 1 K
    description='Temperature (K)',
    disabled=False,
    readout_format='d',
    layout=Layout(width='11cm',
                 margin='5pt')
)
    
temp_slider.style.description_width='3cm'

temp_cb = widgets.Checkbox(
    value=False,
    disabled=True,
    description='Temperature(K)?',
    layout=Layout(width='6cm',
                 margin='5pt')
)
temp_cb.style.description_width='0pt'

def temp_cb_onclick(b):
    temp_slider.disabled = (not temp_cb.value)
temp_cb.observe(temp_cb_onclick)

    
temp5_bt = widgets.Button(
    description='5K',
    style={"button_width": "125px"}
)

temp77_bt = widgets.Button(
    description='77K',
    style={"button_width": "125px"}
)

temp298_bt = widgets.Button(
    description='298K',
    style={"button_width": "125px"}
)

temp5_bt.style.description_width='0pt'
temp77_bt.style.description_width='0pt'
temp298_bt.style.description_width='0pt'

def temp5_bt_onclick(b):
    temp_slider.value = 5
temp5_bt.on_click(temp5_bt_onclick)

def temp77_bt_onclick(b):
    temp_slider.value = 77
temp77_bt.on_click(temp77_bt_onclick)

def temp298_bt_onclick(b):
    temp_slider.value = 298
temp298_bt.on_click(temp298_bt_onclick)

def temp_slider_onclick(b):
    temp_slider.disabled = (not temp_slider.value)
temp_slider.observe(temp_slider_onclick)


##########################################################
## Temperature Slider & Buttons
##########################################################


calc_button = widgets.Button(
    description='Calculate',
    disabled=True,
    tooltip='Click to calculate spectra',
)


def calc_button_onclick(b):
    global Freqs
    
    if export_cb.value==True and export_prefix_is_valid()==False:
        setmsg('Please select a valid file name or deselect \'Save on finish?\'')
        return
    Nsites = len(Freqs)
    for n in range(0, Nsites):
        Freqs[n] = FreqBoxList[n].value
    setmsg('')
    calculate_spec()
    if export_cb.value==True:
        export_button_onclick(0)
            
calc_button.on_click(calc_button_onclick)

progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=1,
    step=1,
    description='',
    bar_style='success', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal',
    layout=Layout(width='7cm')
)


# Homogeneous Profile

gauss_slider = widgets.IntSlider(
    value=10,
    min=5, # min value
    max=300, # max value
    step=1, # exponent step
    description='FWHM (1/cm)',
    disabled=False,
    readout_format='d',
    layout=Layout(width='11cm',
                 margin='5pt')
)
gauss_slider.style.description_width='3cm'

gauss_fwhm = int(round(gauss_slider.value))
def gauss_fwhm_on_change(v):
    global gauss_fwhm
    gauss_fwhm = int(round(v['new']))
gauss_slider.observe(gauss_fwhm_on_change, names='value')

gauss_cb = widgets.Checkbox(
    value=True,
    disabled=False,
    description='Gaussian Convolution?',
    layout=Layout(width='6cm',
                 margin='5pt')
)
gauss_cb.style.description_width='0pt'

def gauss_cb_onclick(b):
    gauss_slider.disabled = (not gauss_cb.value)
gauss_cb.observe(gauss_cb_onclick)

GaussBox = widgets.Box([gauss_cb, gauss_slider], 
                         layout=Layout(flex_flow='column',
                                    width=grid_total_width))



####################################################################
# Phonon Spectral Density
####################################################################


# sphon_slider = widgets.FloatSlider(
#     value=0.8,
#     min=0.0, # min value
#     max=2.0, # max value
#     step=0.01, # exponent step
#     description='S Factor',
#     disabled=False,
#     readout_format='.2f',
#     layout=Layout(width='11cm',
#                  margin='5pt')
# )
# sphon_slider.style.description_width='3cm'

# Sphon = sphon_slider.value
# def sphon_slider_on_change(v):
#     global Sphon
#     Sphon = v['new']
# sphon_slider.observe(sphon_slider_on_change, names='value')

# phonon_cb = widgets.Checkbox(
#     value=True,
#     disabled=False,
#     description='Phonon Convolution?',
#     layout=Layout(width='6cm',
#                  margin='5pt')
# )
# phonon_cb.style.description_width='0pt'

# def phonon_cb_onclick(b):
#     sphon_slider.disabled = (not phonon_cb.value)
# phonon_cb.observe(phonon_cb_onclick)

# PhonBox = widgets.Box([phonon_cb, sphon_slider], 
#                          layout=Layout(flex_flow='column',
#                                     width=grid_total_width))



# Dropdowns
def_phonon_drop = widgets.Dropdown(
    options=[],
    description='',
    disabled=True,
    layout = Layout(width='3cm')
)

cla_phonon_drop = widgets.Dropdown(
    options=[],
    description='',
    disabled=True,
    layout = Layout(width='3cm')
)

clb_phonon_drop = widgets.Dropdown(
    options=[],
    description='',
    disabled=True,
    layout = Layout(width='3cm')
)

bca_phonon_drop = widgets.Dropdown(
    options=[],
    description='',
    disabled=True,
    layout = Layout(width='3cm')
)

# S float boxes
def_phonon_s = widgets.BoundedFloatText(
    value=0.5,
    min=0.0,
    max=10.0,
    step=0.01,
    description='',
    disabled=True,
    layout = Layout(width='2cm')
)

cla_phonon_s = widgets.BoundedFloatText(
    value=0.5,
    min=0.0,
    max=10.0,
    step=0.01,
    description='',
    disabled=True,
    layout = Layout(width='2cm')
)

clb_phonon_s = widgets.BoundedFloatText(
    value=0.5,
    min=0.0,
    max=10.0,
    step=0.01,
    description='',
    disabled=True,
    layout = Layout(width='2cm')
)

bca_phonon_s = widgets.BoundedFloatText(
    value=0.5,
    min=0.0,
    max=10.0,
    step=0.01,
    description='',
    disabled=True,
    layout = Layout(width='2cm')
)

# Load profile from file and interpolate to current vaxis resolution
def get_spd(fname):
    dat = np.loadtxt('misc/Phonons/' + fname + '.spd')
    spd = np.interp(vaxis-vaxis[0], dat[:,0], dat[:,1])
    return spd

# Return S value to 3 decimal points
def get_sphon(fname):
    spd = get_spd(fname)
    return np.sum(spd)*(vaxis[1]-vaxis[0])

# On-click methods
def clb_phonon_drop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        clb_phonon_s.value = get_sphon(clb_phonon_drop.value)
clb_phonon_drop.observe(clb_phonon_drop_on_change)

def def_phonon_drop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        def_phonon_s.value = get_sphon(def_phonon_drop.value)
def_phonon_drop.observe(def_phonon_drop_on_change)

def cla_phonon_drop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        cla_phonon_s.value = get_sphon(cla_phonon_drop.value)
cla_phonon_drop.observe(cla_phonon_drop_on_change)

def bca_phonon_drop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        bca_phonon_s.value = get_sphon(bca_phonon_drop.value)
bca_phonon_drop.observe(bca_phonon_drop_on_change)


# Routine to get possible mode lists
def get_modes_list():
    out = !{"ls ./misc/Phonons/*.spd"}
    phononList = []
    for line in out:
        if line[-4:]=='.spd':
            phononList.append(line[:-4].split('/')[-1])
    def_phonon_drop.options = phononList
    cla_phonon_drop.options = phononList
    clb_phonon_drop.options = phononList
    bca_phonon_drop.options = phononList
    if len(phononList)>0:
        def_phonon_drop.value = phononList[0]
        cla_phonon_drop.value = phononList[0]
        clb_phonon_drop.value = phononList[0]
        bca_phonon_drop.value = phononList[0]

# Check box to indicate whether to use phonon convolution
phonon_cb = widgets.Checkbox(
    value=True,
    disabled=False,
    description='Phonon Sideband Convolution?',
    layout=Layout(width='8cm',
                 margin='5pt')
)
phonon_cb.style.description_width='0pt'

# On-click method for turning phononrations on or off. 
def phonon_cb_onclick(b):
    
    drop_list = [cla_phonon_drop, clb_phonon_drop, bca_phonon_drop, def_phonon_drop]
    sphonon_list = [cla_phonon_s, clb_phonon_s, bca_phonon_s, def_phonon_s]
    keys = ['CLA', 'CLB', 'BCA', 'UNK']
    
    # Disable everything
    for drop in drop_list:
        drop.disabled = True

    for sphonon in sphonon_list:
        sphonon.disabled = True

    # Now, if phononrations are being enabled, re-enable the relevant ones
    if phonon_cb.value==True:
        for name in Names:
            for k in range(0, len(keys)):
                if name[0:3].lower()==keys[k].lower():
                    drop_list[k].disabled = False
                    sphonon_list[k].disabled = False

        active_drops = [drop for drop in drop_list if drop.disabled==False]
        active_sphonons = [sphonon for sphonon in sphonon_list if sphonon.disabled==False]

        if len(active_drops)==0:
            drop_list[-1].disabled = False
            sphonon_list[-1].disabled = False

        for drop in drop_list:
            ndx = drop_list.index(drop)

            # If pigment type doesn't already match, update
            if drop.value[0:3].lower!=keys[ndx][0:3].lower:
                for opt in drop.options:
                    if opt[0:3].lower()==keys[ndx][0:3].lower():
                        drop.value = opt
                        sphonon_list[ndx].value = get_sphon(drop.value)

                        # If we found a match, stop looking
                        break
    
phonon_cb.observe(phonon_cb_onclick)

# Initialize display
get_modes_list()
phonon_place_holder = widgets.Label(value='', layout=Layout(width='1.3cm'))
phonon_mode_label = widgets.HTML('<b>Mode Density</b>', layout=Layout(width='3cm'))
phonon_s_label = widgets.HTML('<b>S<sub>tot</sub></b>')

PhononsBox = widgets.Box([phonon_cb, 
                      widgets.Box([phonon_place_holder, phonon_mode_label, phonon_s_label]),
                      widgets.Box([widgets.HTML('CLA: ', layout=Layout(width='1cm')),
                                   cla_phonon_drop, cla_phonon_s]),
                      widgets.Box([widgets.HTML('CLB: ', layout=Layout(width='1cm')),
                                   clb_phonon_drop, clb_phonon_s]),
                      widgets.Box([widgets.HTML('BCA: ', layout=Layout(width='1cm')), 
                                   bca_phonon_drop, bca_phonon_s]),
                      widgets.Box([widgets.HTML('Other:', layout=Layout(width='1cm'))
                                   , def_phonon_drop, def_phonon_s])
                     ],
                         layout=Layout(flex_flow='column',
                                    width=grid_total_width))


####################################################################
# Local Vibrational Spectral Density
####################################################################


# Dropdowns
def_vib_drop = widgets.Dropdown(
    options=[],
    description='',
    disabled=True,
    layout = Layout(width='3cm')
)

cla_vib_drop = widgets.Dropdown(
    options=[],
    description='',
    disabled=True,
    layout = Layout(width='3cm')
)

clb_vib_drop = widgets.Dropdown(
    options=[],
    description='',
    disabled=True,
    layout = Layout(width='3cm')
)

bca_vib_drop = widgets.Dropdown(
    options=[],
    description='',
    disabled=True,
    layout = Layout(width='3cm')
)

# S float boxes
def_vib_s = widgets.BoundedFloatText(
    value=0.5,
    min=0.0,
    max=10.0,
    step=0.01,
    description='',
    disabled=True,
    layout = Layout(width='2cm')
)

cla_vib_s = widgets.BoundedFloatText(
    value=0.5,
    min=0.0,
    max=10.0,
    step=0.01,
    description='',
    disabled=True,
    layout = Layout(width='2cm')
)

clb_vib_s = widgets.BoundedFloatText(
    value=0.5,
    min=0.0,
    max=10.0,
    step=0.01,
    description='',
    disabled=True,
    layout = Layout(width='2cm')
)

bca_vib_s = widgets.BoundedFloatText(
    value=0.5,
    min=0.0,
    max=10.0,
    step=0.01,
    description='',
    disabled=True,
    layout = Layout(width='2cm')
)

# Return total Svib to 3 decimal places
def get_svib(fname):
    return np.sum(np.loadtxt('misc/LocalVibs/' + fname + '.vib')[:,1])

# On-click methods
def clb_vib_drop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        clb_vib_s.value = get_svib(clb_vib_drop.value)
clb_vib_drop.observe(clb_vib_drop_on_change)

def def_vib_drop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        def_vib_s.value = get_svib(def_vib_drop.value)
def_vib_drop.observe(def_vib_drop_on_change)

def cla_vib_drop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        cla_vib_s.value = get_svib(cla_vib_drop.value)
cla_vib_drop.observe(cla_vib_drop_on_change)

def bca_vib_drop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        bca_vib_s.value = get_svib(bca_vib_drop.value)
bca_vib_drop.observe(bca_vib_drop_on_change)


# Routine to get possible mode lists
def get_modes_list():
    out = !{"ls ./misc/LocalVibs/*.vib"}
    vibList = []
    for line in out:
        if line[-4:]=='.vib':
            vibList.append(line[:-4].split('/')[-1])
    def_vib_drop.options = vibList
    cla_vib_drop.options = vibList
    clb_vib_drop.options = vibList
    bca_vib_drop.options = vibList
    if len(vibList)>0:
        def_vib_drop.value = vibList[0]
        cla_vib_drop.value = vibList[0]
        clb_vib_drop.value = vibList[0]
        bca_vib_drop.value = vibList[0]

# Check box to indicate whether to use local vibrations
vib_cb = widgets.Checkbox(
    value=True,
    disabled=False,
    description='Local Vibrational Convolution?',
    layout=Layout(width='8cm',
                 margin='5pt')
)
vib_cb.style.description_width='0pt'

# On-click method for turning vibrations on or off. 
def vib_cb_onclick(b):
    
    drop_list = [cla_vib_drop, clb_vib_drop, bca_vib_drop, def_vib_drop]
    svib_list = [cla_vib_s, clb_vib_s, bca_vib_s, def_vib_s]
    keys = ['CLA', 'CLB', 'BCA', 'UNK']
    
    # Disable everything
    for drop in drop_list:
        drop.disabled = True

    for svib in svib_list:
        svib.disabled = True

    # Now, if vibrations are being enabled, re-enable the relevant ones
    if vib_cb.value==True:
        for name in Names:
            for k in range(0, len(keys)):
                if name[0:3].lower()==keys[k].lower():
                    drop_list[k].disabled = False
                    svib_list[k].disabled = False

        active_drops = [drop for drop in drop_list if drop.disabled==False]
        active_svibs = [svib for svib in svib_list if svib.disabled==False]

        if len(active_drops)==0:
            drop_list[-1].disabled = False
            svib_list[-1].disabled = False

        for drop in drop_list:
            ndx = drop_list.index(drop)

            # If pigment type doesn't already match, update
            if drop.value[0:3].lower!=keys[ndx][0:3].lower:
                for opt in drop.options:
                    if opt[0:3].lower()==keys[ndx][0:3].lower():
                        drop.value = opt
                        svib_list[ndx].value = np.sum(np.loadtxt('misc/LocalVibs/' + drop.value + '.vib')[:,1])

                        # If we found a match, stop looking
                        break
    
vib_cb.observe(vib_cb_onclick)

# Initialize display
get_modes_list()
vib_place_holder = widgets.Label(value='', layout=Layout(width='1.3cm'))
vib_mode_label = widgets.HTML('<b>Mode Density</b>', layout=Layout(width='3cm'))
vib_s_label = widgets.HTML('<b>S<sub>tot</sub></b>')

VibBox = widgets.Box([vib_cb, 
                      widgets.Box([vib_place_holder, vib_mode_label, vib_s_label]),
                      widgets.Box([widgets.HTML('CLA: ', layout=Layout(width='1cm')),
                                   cla_vib_drop, cla_vib_s]),
                      widgets.Box([widgets.HTML('CLB: ', layout=Layout(width='1cm')),
                                   clb_vib_drop, clb_vib_s]),
                      widgets.Box([widgets.HTML('BCA: ', layout=Layout(width='1cm')), 
                                   bca_vib_drop, bca_vib_s]),
                      widgets.Box([widgets.HTML('Other:', layout=Layout(width='1cm'))
                                   , def_vib_drop, def_vib_s])
                     ],
                         layout=Layout(flex_flow='column',
                                    width=grid_total_width))


################################################################
# Disorder
################################################################

# Disorder
nit_slider = widgets.FloatLogSlider(
    value=1000,
    base=10,
    min=0, # max exponent of base
    max=6, # min exponent of base
    step=1, # exponent step
    description='Iterations',
    disabled=False,
    readout_format='d',
    layout=Layout(width='11cm',
                 margin='5pt')
)


specfilelbl = widgets.Label(value='Output file prefix: ')

specfile = widgets.Text(
    value='test',
    placeholder='file prefix',
    layout = widgets.Layout(width='5cm'),
    disabled=False
)

export_cb = widgets.Checkbox(
    value=False,
    disabled=False,
    description='Save on finish?',
    layout=Layout(width='3.5cm')
)
export_cb.style.description_width='0pt'

exportover_cb = widgets.Checkbox(
    value = False,
    disabled = False,
    description = 'Overwrite Existing?',
    layout = widgets.Layout(width='4cm')
)
exportover_cb.style.description_width='0pt'

export_button = widgets.Button(
    description='Save Now',
    layout=Layout(width='3cm'))

def export_prefix_is_valid():
    prefix = specfile.value
    fbase = DATADIR + "/spec/" + prefix
    
    Suffixes = ['_abs.txt', '_cd.txt']
    for suf in Suffixes:
        fname = fbase + suf
        for chars in forbidden_strings:
            if len(prefix)==0 or prefix.find(chars)!=-1:
                setmsg('Please enter a valid file prefix for output. Avoid spaces and the special characters "~", "..", "/", and "\\".')
                return False

        if os.path.isfile(fname)==True:
            if exportover_cb.value==False:
                setmsg("Output file exists. Please check 'Overwrite Existing' or pick another export name.")
                return False
            else:
                !{"rm " + fname}
            
    return fbase

def export_button_onclick(b):
    fbase = export_prefix_is_valid()
    if fbase!=False:
        np.savetxt(fbase + '_abs.txt', np.vstack([vaxis, abs_line.get_ydata()]).T)
        np.savetxt(fbase + '_cd.txt', np.vstack([vaxis, cd_line.get_ydata()]).T)
        np.savetxt(fbase + '_fl.txt', np.vstack([vaxis, flu_line.get_ydata()]).T)
        setmsg('Spectrum exported under prefix ' + fbase)
export_button.on_click(export_button_onclick)


LblList = []
FreqBoxList = []
FWHMBoxList = []
PigBoxList = []

Htemp = widgets.HBox([temp5_bt, temp77_bt, temp298_bt])
TempBox = widgets.Box([Htemp, temp_slider], 
                         layout=Layout(flex_flow='column',
                                    width=grid_total_width,
                                    border='solid 0.5pt gray'))







# SetDescript = widgets.HTML('<b style="font-size:12pt">Set Defaults</b>')

Nits = int(round(nit_slider.value))
def nit_on_change(v):
    global Nits
    Nits = int(round(v['new']))
nit_slider.observe(nit_on_change, names='value')

disorder_cb = widgets.Checkbox(
    value=True,
    disabled=False,
    description='Gaussian Disorder?',
    layout=Layout(width='6cm',
                 margin='5pt')
)
disorder_cb.style.description_width='0pt'



def dis_cb_onclick(b):
    nit_slider.disabled = (not disorder_cb.value)
disorder_cb.observe(dis_cb_onclick)

DisorderBox = widgets.Box([disorder_cb, nit_slider], 
                         layout=Layout(flex_flow='column',
                                    width=grid_total_width))

def fwhm_onclick(b):
    global FWHMBoxList
    for widg in FWHMBoxList:
        widg.value = fwhm_value.value
    
fwhm_button = widgets.Button(
    description='Set all',
    layout=Layout(width='2cm')
)
fwhm_button.on_click(fwhm_onclick)

fwhm_value = widgets.BoundedFloatText(
    description='',
    value=200.0,
    min=0.0,
    step=1.0,
    max=2000.0,
    layout=Layout(width='2cm')
)
fwhm_value.observe(fwhm_onclick, 'value')
# fwhm_value.on_submit(fwhm_onclick)

fwhm_lbl = widgets.Label(
    value='FWHM (1/cm):',
    layout=Layout(width='2.5cm')
)

FWHMDefBox = widgets.Box([fwhm_lbl, fwhm_value, fwhm_button], 
                     layout=Layout(
                         flex_flow = 'row',
                         margin='0.25cm'))

def freq_onclick(b):
    global FreqBoxList
    for widg in FreqBoxList:
        widg.value = freq_value.value
    
freq_button = widgets.Button(
    description='Set all',
    layout=Layout(width='2cm')
)
freq_button.on_click(freq_onclick)

freq_value = widgets.BoundedFloatText(
    description='',
    value=14925.0,
    min=0.0,
    step=1.0,
    max=20000.0,
    layout=Layout(width='2cm')
)
freq_value.observe(freq_onclick, 'value')

freq_lbl = widgets.Label(
    value='Center (1/cm):',
    layout=Layout(width='2.5cm')
)

FreqDefBox = widgets.Box([freq_lbl, freq_value, freq_button], 
                     layout=Layout(
                         flex_flow = 'row',
                         margin='0.25cm'))

# SetDefBox = widgets.Box([DisorderBox, FWHMDefBox], 
#                        layout=Layout(
#                            flex_flow='column',
# #                            margin='0.25cm',
# #                            border='solid',
#                            width=grid_total_width
#                        ))
spec_down = widgets.HTML(HTMLButtonPrompt.format(link=DATADIR + "/spec/", text='Download Spectra'),
        layout=Layout(
            margin='10pt'
        ))

spec_process = widgets.HTML(HTMLButtonPrompt.format(link='Calculator.ipynb', text='Process Spectra'),
        layout=Layout(
            margin='10pt'
        ))


#######################################################################
# Param box
#######################################################################

PigHead = widgets.Box([
    #widgets.Label(value="Pigment:", layout=Layout(width='2cm')),
    widgets.HTML(value="<center><b style='font-size:12pt'>Pigment</b></center>", layout=Layout(width=grid_width)),
    widgets.HTML(value="<center><b style='font-size:12pt'>Frequency</b></center>", layout=Layout(width=grid_width)),
    widgets.HTML(value="<center><b style='font-size:12pt'>FHWM</b></center>", layout=Layout(width=grid_width)),
    ], layout=Layout(flex='row', grid_gap=ggap))

DescriptText ="<br><p><b>Frequency</b> is the site energy center value in cm<sup>-1</sup>.</p>"\
    "<p><b>FWHM</b> is the site energy distribution width in cm<sup>-1</sup>.</p>"
PigDescript = widgets.HTML(DescriptText)


PigBox = widgets.Box([DisorderBox, FreqDefBox, FWHMDefBox, PigHead] + PigBoxList + [PigDescript],
                      layout=Layout(flex_flow='column',
#                                    border='solid 0.5pt',
                                   width=grid_total_width))


#######################################################################
# Display box
#######################################################################

norm_drop = widgets.Dropdown(
    value = 'Absolute Max',
    options=['Absolute Area', 'Absolute Max'],
    description='Normalize spectra by:',
    disabled=False,
    layout = Layout(width='8cm')
)
norm_drop.style.description_width='3.4cm'
def norm_drop_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        update_spec_lims()
norm_drop.observe(norm_drop_on_change)

abs_cb = widgets.Checkbox(
    value=True,
    description='Abs.',
    disabled=False,
    layout=Layout(width='2cm')
)
abs_cb.style.description_width='0cm'
flu_cb = widgets.Checkbox(
    value=True,
    description='Flu.',
    disabled=False,
    layout=Layout(width='2cm')
)
flu_cb.style.description_width='0cm'
cd_cb = widgets.Checkbox(
    value=True,
    description='CD',
    disabled=False,
    layout=Layout(width='2cm')
)
cd_cb.style.description_width='0cm'

def spec_cb_on_click(b):
    cb_list = [abs_cb, flu_cb, cd_cb]
    line_list = [abs_line, flu_line, cd_line]
    linx_list = [abx_line, flx_line, cdx_line]
    for n in range(0, len(cb_list)):
        line_list[n].set_visible(cb_list[n].value)
    update_spec_lims()
    
flu_cb.observe(spec_cb_on_click)
abs_cb.observe(spec_cb_on_click)
cd_cb.observe(spec_cb_on_click)


abslab = widgets.Label(value='Abs. Reference:', layout=Layout(width='2.5cm'))
absup = widgets.FileUpload(
    accept='',  # Accepted file extension
    multiple=False,  # True to accept multiple files upload else False
    layout=Layout(width='3cm')
)
absdel = widgets.Button(
    description='Delete',
    layout=Layout(width='1.75cm')
)

cdlab = widgets.Label(value='CD Reference:', layout=Layout(width='2.5cm'))
cdup = widgets.FileUpload(
    accept='',  # Accepted file extension
    multiple=False,  # True to accept multiple files upload else False
    layout=Layout(width='3cm')
)
cddel = widgets.Button(
    description='Delete',
    layout=Layout(width='1.75cm')
)

flulab = widgets.Label(value='Flu. Reference:', layout=Layout(width='2.5cm'))
fluup = widgets.FileUpload(
    accept='',  # Accepted file extension
    multiple=False,  # True to accept multiple files upload else False
    layout=Layout(width='3cm')
)
fludel = widgets.Button(
    description='Delete',
    layout=Layout(width='1.75cm')
)

def norm_spec(yvals):
    dat = yvals.copy()
    if norm_drop.value=='Absolute Max':
        mx = np.max(np.abs(dat))
        if mx!=0:
            dat /= mx
    if norm_drop.value=='Absolute Area':
        fac = np.sum(np.abs(dat))
        if fac!=0:
            dat /= fac
    return dat

def add_ref(dat, flag):
    # If the largest value in the axis is less than 2000, 
    # we'll assume this is a wavelength axis in nm and convert
    # to cm-1. 
    if np.max(dat[:,0])<2000.0:
        dat[:,0] = 1.0e+7/dat[:,0]
    if dat[1,0]-dat[0,0]<0:
        dat = dat[::-1,:]
    n1 = np.argmin(np.abs(vaxis-dat[0,0]))
    n2 = np.argmin(np.abs(vaxis-dat[-1,0]))
    ydat = np.zeros(np.shape(vaxis))
    ydat[n1:n2] = np.interp(vaxis[n1:n2], dat[:,0], dat[:,1])
    ydat = norm_spec(ydat)
    
    if flag=='abs':
        abx_line.set_ydata(ydat)
    if flag=='flu':
        flx_line.set_ydata(ydat)
    if flag=='cd':
        cdx_line.set_ydata(ydat)
    
    update_spec_lims()
    specfig.canvas.draw()
    specfig.canvas.flush_events()

def ref_spec_update(widg, flag):
    for item in widg.value:
        fname = item
    try:
        dat = widg.value[fname]["content"]
        fx = io.BytesIO(dat)
        add_ref(np.loadtxt(fx), flag)
        fx.close()
    except:
        print('Could not load data from file. Please check file format')

    
    
def absup_on_value_change(change):
    ref_spec_update(absup, 'abs')
def fluup_on_value_change(change):
    ref_spec_update(fluup, 'flu')
def cdup_on_value_change(change):
    ref_spec_update(cdup, 'cd')
        
absup.observe(absup_on_value_change, 'value')
fluup.observe(fluup_on_value_change, 'value')
cdup.observe(cdup_on_value_change, 'value')

DispBox = widgets.Box([
                    widgets.Box([widgets.Label(value='Show: '), abs_cb, flu_cb, cd_cb],
                                   layout=Layout(flex='row')),
                    norm_drop,
                    widgets.Box([abslab, absup, absdel],
                          layout=Layout(flex_flow='row')),
                    widgets.Box([flulab, fluup, fludel],
                          layout=Layout(flex_flow='row')),
                    widgets.Box([cdlab, cdup, cddel],
                          layout=Layout(flex_flow='row')),
                      ],
                      layout=Layout(flex_flow='column',
                                    align_items='flex-end',
                                   width='9cm'))

ExportBox = widgets.Box([
    widgets.HBox([specfilelbl, specfile, export_button]),
    widgets.HBox([export_cb, exportover_cb]),
    widgets.HBox([spec_down, spec_process]),
    ],
    layout=Layout(flex_flow='column',
                align_items='flex-end',
               width='9cm')
)
ParamAcc = widgets.Accordion(children=[PigBox, 
                                       GaussBox,
                                       PhononsBox, 
                                       VibBox,
                                       DispBox,
                                       ExportBox
                                      ],
                            selected_index=None, 
                            layout=Layout(width=grid_total_width))

ParamAcc.set_title(0, 'Site Energies & Disorder')
ParamAcc.set_title(1, 'Gaussian Smoothing')
ParamAcc.set_title(2, 'Phonon Profile')
ParamAcc.set_title(3, 'Local Vibrations')
ParamAcc.set_title(4, 'Display & Comparison')
ParamAcc.set_title(5, 'Export & Processing')

msglbl = widgets.HTML(value='', layout=Layout(width=grid_total_width))

def setmsg(txt):
    msglbl.value = "<p style=\"font-size:12pt;\">" + txt + "</p>"
    
big_bt = widgets.Button(
    description='Make Big'
)

display(widgets.Box([
    widgets.Box([widgets.HBox([load_button, param_drop, refresh_button]),
                msglbl,
                 widgets.HBox([big_bt], layout=Layout(flex='row')),
                specout,
                 ], layout=Layout(
                    flex_flow='column',
                    align_items='flex-start',
                )),
        widgets.Box([TempBox,
                widgets.HBox([calc_button, progress_bar]),
                     widgets.Label(layout=Layout(height='15px')),
                    ParamAcc,
                    ], layout=Layout(flex_flow='column', align_items='flex-start'))
], layout=Layout(flex='row')))

small_fig_size = (4,3.5)
big_fig_size = (6,5)

with specout:
    %matplotlib notebook
    specfig = plt.figure(figsize=small_fig_size)
    specax = plt.gca()
    specax2 = specax.twiny()
    abx_line, = specax.plot(vaxis, 0.0*vaxis, 'b--')
    cdx_line, = specax.plot(vaxis, 0.0*vaxis, 'g--')
    flx_line, = specax.plot(vaxis, 0.0*vaxis, 'r--')
    abs_line, = specax.plot(vaxis, 0.0*vaxis, 'b')
    cd_line, = specax.plot(vaxis, 0.0*vaxis, 'g')
    flu_line, = specax.plot(vaxis, 0.0*vaxis, 'r')
    build_axes()

def make_small(b):
    specout.layout=Layout(width=str(small_fig_size[0])+'in', height=str(small_fig_size[1])+'in')
    specfig.set_size_inches(small_fig_size[0]*1.4, small_fig_size[1]*1.3)
    big_bt.description='Make Big'

def make_big(b):
    specout.layout=Layout(width=str(big_fig_size[0])+'in', height=str(big_fig_size[1])+'in')
    specfig.set_size_inches(big_fig_size[0]*1.4, big_fig_size[1]*1.2)
    big_bt.description='Make Small'

def toggle_size(b):
    if big_bt.description=='Make Big':
        make_big(0)
    elif big_bt.description=='Make Small':
        make_small(0)
big_bt.on_click(toggle_size)
    

Box(children=(Box(children=(HBox(children=(Button(description='Load Model', layout=Layout(width='2.5cm'), styl…